In [ ]:
# Will Bring the Dataset in the form tag: {Tokenized Sentences}

In [2]:
#  DATA PREPROCESSING CELL  #
#===========================#

import pandas as pd
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('english')


pd.set_option('display.max_colwidth', -1)


# Path to Initial Dataset
path = './quotes.json'

# Import JSON as Pandas Dataframe
df = pd.read_json(path, orient='columns')
# df.head(5)

# Remove list for all useless Tags
remove_list = ['attributed-no-source']

# Function to Remove Redundant Columns and Tags 
def curate_data(data):    
    data = data.drop(['Author', 'Popularity', 'Category'], axis = 1)
    data.drop_duplicates(subset ="Quote", keep = 'first', inplace = True) 
    data['Tags'] = data['Tags'].apply(lambda tags_list: [x for x in tags_list if x not in remove_list and not x.startswith('misattributed')])        
    return data


# Function to Apply Text Preprocessing
def preprocess(sentence):
    return [_ for _ in word_tokenize(sentence.lower().translate(str.maketrans('', '', string.punctuation))) if _ not in stop]




In [9]:
# Remove Irrelevant Columns and Entries
curated_data = curate_data(df)
curated_data.index = range(len(curated_data))

# # Write Curated Data to JSON
# curated_data.to_json(r'./curated-data.json')

# Preprocessing the dataset
curated_data.Quote = curated_data.Quote.apply(preprocess)
curated_data = curated_data.explode('Tags').reset_index(drop=True)
curated_data.head(50)


# testdf['Quote'].apply(lambda x: x.lower()).apply(lambda x: nltk.word_tokenize(x)).head()

,Quote,Tags
0,"[dont, cry, smile, happened]",cry
1,"[dont, cry, smile, happened]",crying
2,"[dont, cry, smile, happened]",experience
3,"[dont, cry, smile, happened]",happiness
4,"[dont, cry, smile, happened]",joy
5,"[dont, cry, smile, happened]",life
6,"[dont, cry, smile, happened]",optimism
7,"[dont, cry, smile, happened]",sadness
8,"[dont, cry, smile, happened]",smile
9,"[dont, cry, smile, happened]",smiling


In [11]:
from functools import reduce
# Identify Unique Tags
# Create flat list of Tags
tags = testdf['Tags'].tolist()
flat_tags = list(dict.fromkeys(tags))


# Sort the Dataset by Tags in a Dictionary 
def create_dict(data):
    return {tag: list(set(reduce(lambda x, y: x + y, data.Quote[data.Tags == tag]))) for tag in flat_tags}


a = create_dict(curated_data)
print(a)

# Making sure there are no duplicates
[type(_) for _ in a.values()]
assert all([type(x) == list for x in a.values()])

for i, d in enumerate(a.items()):
    if len(d[1]) != len(set(d[1])):
        print(i, d[0], sorted(d[1]), len(d[1]), "G", len(set(d[1])), set(d[1]))
assert all([len(x) == len(set([_ for _ in x])) for x in a.values()])

# Dataset ready to be passed to Convnet


{'cry': ['could', 'solve', 'funeral', 'sing', 'happiness', 'tomorrow', 'later', 'remind', 'god', 'seeds', 'long', 'face', 'smile', 'stirred', 'least', 'bador', 'waking', 'succeed', 'perhaps', 'pain—and', 'youcry', '11612', 'speaki', 'call', 'yesterdayi', 'wouldnt', 'yesterdayto', 'despite', 'response', 'mountains', 'lasts', 'happened', 'hard', 'fountainslaugh', 'forgive', 'sooner', 'turned', 'water', 'cut', 'heavenly', 'laugh', 'angels', 'tomorrowsimply', 'found', 'divinesing', 'struggle', 'tears', 'today', 'needed', 'half', 'problems', 'wildest', 'lessons', 'aching', 'mind', 'four', 'life', 'remember', 'creatures', 'always', 'resolve', 'spoke', 'think', 'put', 'stay', 'anything', 'every', 'someday', 'sleeping', 'force', 'cryi', 'cry', 'hands', 'bareif', 'sadness', 'selfpity', 'one', 'way', 'shone', 'backthe', 'others', 'future', 'move', 'thought', 'lord', 'making', 'see', 'dont', 'stop', 'reflectthe', 'problem', 'moment', 'right', 'fail', 'let', 'niv', 'safe', 'theyll', 'mador', 'crie

In [ ]:
# import shorttext
# #  TRAIN DATA LOADING CELL  #
# #===========================#

# # Subject Dataset
# trainclassdict = shorttext.data.subjectkeywords()
# print(trainclassdict)


# NIH RePORT Dataset
#trainclassdict = shorttext.data.nihreports()
#print(trainclassdict)

# Inaugural Addresses Dataset
#trainclassdict = shorttext.data.inaugural()


# User Defined Dataset (JSON)
#trainclassdict = shorttext.data.retrieve_jsondata_as_dict('./quotes.json')
#print(trainclassdict[0])
# User Defined Dataset (CSV Heading & at least 2 columns)
# trainclassdict = shorttext.data.retrieve_csvdata_as_dict('/path/to/file.csv')

